### Imports

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

In [2]:
%matplotlib inline

### Load data

In [3]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


### Pick some hyperparameters

In [4]:
learning_rate = 0.001
num_steps = 500
batch_size = 128
display_step = 10

### Pick the size of input features, number of output classes and dropout rate

In [5]:
num_input = 28 * 28 
num_classes = 10 
dropout_rate = 0.9

### Initialize input for tf.Graph()

In [6]:
X = tf.placeholder(tf.float32, [None, num_input])
Y = tf.placeholder(tf.float32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32) 

### Create sample convolutional & max-pool layers 

In [7]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

### Create a net with two convolutional layers and one fully connected layer

In [8]:
def conv_net(x):

    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)

    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fcl = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fcl = tf.add(tf.matmul(fcl, weights['wd1']), biases['bd1'])
    fcl = tf.nn.relu(fcl)
    
    # Dropout
    fcl = tf.nn.dropout(fcl, dropout_rate)

    # Output, class prediction
    out = tf.add(tf.matmul(fcl, weights['out']), biases['out'])
    return out

### Initialize weights and biases

In [9]:
weights = {
    # 5x5 convolutional layer, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 convolutional layer, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected layer, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, num_classes]))}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([num_classes]))}

### Initialize model saver

In [10]:
saver = tf.train.Saver()

### Training 

In [11]:
with tf.Session() as sess:
    
    logits = conv_net(X)
    prediction = tf.nn.softmax(logits)
    
    loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op)
    
    correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    
    sess.run(tf.global_variables_initializer())

    for step in range(1, num_steps+1):
        # Forward step
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Backpropagation step
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y, keep_prob: 1.0})

        if step % display_step == 0 or step == 1:
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x, Y: batch_y,keep_prob: 1.0})
            print(f'Step: {step},  Losses: {loss:.4f}, Train accuracy: {100 * acc:.3f}%')

    # Count accuracy on the training dataset
    test_acc = sess.run(accuracy, feed_dict={X: mnist.test.images[:256], Y: mnist.test.labels[:256], keep_prob: 1.0})
    saver.save(sess, 'pred/model.ckpt')

    print(f'\n\nTesting Accuracy: {100 * test_acc:.2f}%')


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Step: 1,  Losses: 87661.6484, Train accuracy: 6.250%
Step: 10,  Losses: 34683.2188, Train accuracy: 17.969%
Step: 20,  Losses: 18888.7402, Train accuracy: 33.594%
Step: 30,  Losses: 11993.9326, Train accuracy: 53.125%
Step: 40,  Losses: 11223.9707, Train accuracy: 50.781%
Step: 50,  Losses: 7625.2271, Train accuracy: 67.188%
Step: 60,  Losses: 7939.4194, Train accuracy: 68.750%
Step: 70,  Losses: 4660.4380, Train accuracy: 75.000%
Step: 80,  Losses: 5499.3398, Train accuracy: 75.781%
Step: 90,  Losses: 5073.2095, Train accuracy: 78.125%
Step: 100,  Losses: 4498.8184, Train accuracy: 78.125%
Step: 110,  Losses: 5365.1035, Train accuracy: 75.781%
Step: 120,  Losses: 4173.8115, Train accuracy: 75.000%
Step: 130,  Losses: 2407.5586, Train accuracy: 85.156%
Step: 140,  Losses: 2757.9536, Train accuracy: